Author: Dan Shea  
Date: 2019.08.08  

# VCFstats
`VCFstats` will load a VCF file as a `pandas` dataframe and calculate statistical metrics that may be of use when examining variant data.

In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import scipy.stats as stats
import os.path
import os
import re
from collections import OrderedDict
import gzip
import statsmodels.stats.multitest as multitest

In [2]:
class Error(Exception):
    """Base class for exceptions in this module.
    Attributes:
    expression -- input expression in which the error occurred
    message -- explanation of the error
    """
    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

        
class PloidyError(Error):
    """Exception raised for ploidy errors in the input."""
    pass


class HeterozygousParentError(Error):
    pass


class NoAlleleFrequenciesError(Error):
    pass

        
class VCF:
    def __is_double_hash__(self, line):
        match = re.match('^##', line)
        if match:
            return True
        else:
            return False
        
    def __is_single_hash__(self, line):
        match = re.match('^#[^#]', line)
        if match:
            return True
        else:
            return False
    
    def __is_key_value_pair__(self, line):
        match = re.match('^(.*?)=(.*)$', line)
        if match:
            return True
        else:
            return False
    
    def __isref__(self, val):
        match = re.match('0[|/]0', val)
        if match:
            return True
        else:
            match = re.match('1[|/]1', val)
            if match:
                return False
            else:
                match = re.match('0[|/]1', val)
                if match:
                    raise HeterozygousParentError(val, 'Parent contains a heterozygous call at this loci.')
                else:
                    match = re.match('1[|/]0', val)
                    if match:
                        raise HeterozygousParentError(val, 'Parent contains a heterozygous call at this loci.')
                    else:
                        raise PloidyError(val, 'Genotype does not appear to be diploid.')
    
    def __init__(self, vcf_filename, reference_filename=None):
        # Internal VCF data
        self.__colnames__ = list()
        self.__samples__ = list()
        self.__hasFORMAT__ = False
        self.__expected_frequencies__ = np.array([0.5, 0.5]) # Expected Mendelian frequencies
        # Attributes
        self.meta = OrderedDict()       # meta-data in the VCF (key-value pairs)
        self.data = pd.DataFrame()      # Pandas dataframe of the genotyping data
        self.num_records = 0            # How many records were read in at the time of intialization
        self.num_samples = 0            # How many samples are in the file? (Not counting the parent samples)
        self.vcf_filename = None        # The VCF filename
        self.reference_filename = None  # The reference genome associated with the VCF
        self.seqio = None               # If a reference was given, a SeqIO of the reference
        self.allele_frequencies = None  # Allele frequencies calculated by calling calculate_allele_frequencies() method
        self.dump_prefix = None         # Default filename prefix for the dump() method
        self.chisquare_pvalues = None   # p-values of a chi-square goodness of fit test against calculated allele frequencies
        self.chisquare_qvalues = None   # Bejamini-Hochberg FDR corrected p-values of a chi-square goodness of fit test against
                                        # calculated allele frequencies.
        
        # Parse the VCF
        if os.path.isfile(vcf_filename):
            self.vcf_filename = vcf_filename
            with gzip.open(vcf_filename, 'rt') as vcfh:
                header_was_read = False
                vcf_data = list()
                for line in vcfh:
                    line = line.strip()
                    # Double-hash lines are meta lines that consist of key-value pairs
                    # i.e. - ##key=value
                    # If we didn't already see the header check if this line is a key-value pair
                    if not header_was_read and self.__is_double_hash__(line):
                        line = line.lstrip('#')
                        if self.__is_key_value_pair__(line):
                            match = re.match('^(.*?)=(.*)$', line)
                            self.meta[match.group(0)] = match.group(1)
                    # The single hash line is the header line defining the mandatory eight columns:
                    #  1. CHROM  - chromosome: An identifier from the reference genome
                    #  2. POS    - position: The reference position, with the 1st base having position 1.
                    #  3. ID     - identifier: Semi-colon separated list of unique identifiers where available.
                    #  4. REF    - reference base(s): Each base must be one of A,C,G,T,N (case insensitive).
                    #  5. ALT    - alternate base(s): Comma separated list of alternate non-reference alleles. (A,C,G,T,N,*)
                    #  6. QUAL   - quality: Phred-scaled quality score for the assertion made in ALT.
                    #  7. FILTER - filter status: PASS if this position has passed all filters, i.e. a call is made at
                    #             this position.
                    #  8. INFO   - additional information: (String, no white-space, semi-colons, or equals-signs permitted;
                    #             commas are permitted only as delimiters for lists of values)
                    #  9. FORMAT - If genotype information is present, then the same types of data must be present for all
                    #             samples. First a FORMAT field is given specifying the data types and order (colon-separated
                    #             alphanumeric String).
                    # 10. SAMPLE - ALL columns after these columns are unique sample identifiers.
                    
                    # ***** THE HEADER LINE FIELDS ARE TAB-DELIMITED *****
                    
                    # If we didn't already see the header, check if this line is the header
                    elif not header_was_read and self.__is_single_hash__(line):
                        header_was_read = True
                        line = line.lstrip('#')
                        fields = line.split('\t')
                        if fields[8] == 'FORMAT':
                            self.__hasFORMAT__ = True
                            self.__colnames__ = fields[0:9]
                            self.__samples__ = fields[9:]
                        else:
                            self.__colnames__ = fields[0:8]
                            self.__samples__ = fields[8:]
                        
                        # We subtract 2 here because HITOMEBORE and the FOUNDER are also in the file
                        self.num_samples = len(self.__samples__) - 2 
                    
                    # If we already saw the header, start reading in the data
                    elif header_was_read:
                        fields = line.split('\t')
                        # Convert non-string data to the appropriate type
                        fields[1] = int(fields[1])
                        # Because variant calls are performed with respect to a reference sequence
                        # We must account for whichi parent matches the reference and which differs.
                        # Homozygous Hitomebore (dependent upon whether it matches the REF or the ALT)
                        # Homozygous Founder (dependent upon whether it matches the REF or the ALT)
                        # Heterozygous (0|1) or (1|0)
                        start = None
                        if self.__hasFORMAT__:
                            start = 9
                        else:
                            start = 8
                        # Analyze parents
                        hitomebore = fields[start]
                        fields[start] = 'A'
                        founder = fields[start+1]
                        fields[start+1] = 'B'
                        
                        # But who was phone^H^H^H^H^H ref?
                        if self.__isref__(hitomebore):
                            aref = True
                        else:
                            aref = False
                        
                        # A little rendundant, but I want to ensure there's no weirdness in the founder genotypes either
                        if self.__isref__(founder):
                            aref = False
                        else:
                            aref = True
                        
                        for i in range(start+2, len(fields)):
                            val = fields[i]
                            match = re.match('0[|/]0', val)
                            if match:
                                fields[i] = 'A' if aref else 'B'
                            else:
                                match = re.match('1[|/]1', val)
                                if match:
                                    fields[i] = 'B' if aref else 'A'
                                else:
                                    match = re.match('0[|/]1', val)
                                    if match:
                                        fields[i] = 'H'
                                    else:
                                        match = re.match('1[|/]0', val)
                                        if match:
                                            fields[i] = 'H'
                                        else:
                                            raise PloidyError(val, 'Genotype does not appear to be diploid.')
                                            
                        vcf_data.append(fields)
                self.data = pd.DataFrame(data=vcf_data, columns=self.__colnames__+self.__samples__)
                self.num_records = self.data.shape[0]
        
        # Parse the reference genome (if provided)
        if reference_filename:
            if os.path.isfile(reference_filename):
                seqio = SeqIO.parse(reference_filename)
                self.reference_filename = reference_filename
                self.seqio = seqio
        
        # Define a filename for the dump() method
        _tmp = os.path.basename(self.vcf_filename)
        _tmp = _tmp.split(os.path.extsep)
        _tmp.pop() # remove the gz extension
        _tmp.pop() # remove the vcf extension
        _tmp = os.path.extsep.join(_tmp) # re-join
        self.dump_prefix = os.path.join(os.path.dirname(self.vcf_filename), _tmp)
        del(_tmp)
                     
        # Note: In Python, class constructor __init__(self, ...) does not return anything

    def __calculate_allele_frequency__(self, x):
        obs = np.array([0,0])
        if self.__hasFORMAT__:
            start=11
        else:
            start=10
            
        num_obs = len(range(start, len(x)))
        
        for i in range(start, len(x)):
            if x[i] == 'A':
                obs[0] += 1
            elif x[i] == 'B':
                obs[1] += 1
        return obs
    
    def __run_chisquare__(self, x):
        return stats.chisquare(f_obs=x, f_exp=sum(x)*self.__expected_frequencies__)
    
    ###### Public methods ######
    
    def reset_seqio(self):
        '''reset_seqio() resets the seqio generator back to the beginning of the file
        The seqio attribute is reset, but no value is returned. Access the seqio via its attribute 'seqio'.
        '''
        if os.path.isfile(self.reference_filename):
            seqio = SeqIO.parse(reference_filename)
            self.seqio = seqio
            
    def calculate_allele_frequencies(self):
        '''calculate_allele_frequencies() calculates the frequency of Homozygous parental alleles.
        Heterozygous are not used in chi-square testing but may be inferred from the data.
        Returns the DataFrame which is also accessible as the attribute 'allele_frequencies'.
        '''
        self.allele_frequencies = self.data.apply(self.__calculate_allele_frequency__, axis=1)
        self.allele_frequencies = pd.DataFrame(data=self.allele_frequencies.values.tolist(),
                                               columns=['Homozygous A', 'Homozygous B'])
        return self.allele_frequencies
    
    def chisquare(self):
        '''Perform a chi-square goodness of fit on the observed allele frequencies.
        Return the p-values and save them in the chisquare_pvalues attribute.
        '''
        if self.allele_frequencies is not None:
            self.chisquare_pvalues = pd.DataFrame(self.allele_frequencies.apply(self.__run_chisquare__, axis=1).values.tolist())
            self.allele_frequencies = pd.concat([self.allele_frequencies, self.chisquare_pvalues], axis=1)
            # Now we perform FDR correction
            sig, qvalue = multitest.fdrcorrection(self.chisquare_pvalues.loc[:, 'pvalue'])
            self.chisquare_qvalues = pd.DataFrame(zip(qvalue, sig), columns=['qvalue', 'significant'])
            self.allele_frequencies = pd.concat([self.allele_frequencies, self.chisquare_qvalues], axis=1)
            return self.allele_frequencies
        else:
            raise NoAlleleFrequenciesError(None, 'You must run calculate_allele_frequencies() to perform a chi-square test.')
    
    def dump(self, prefix=None):
        '''dump() creates a tsv consisting of the:
        Genotype (if present)
        Allele frequencies (if present)
        Default output is written to the current vcf path and filename (sans .vcf) with an xlsx extension.
        May be changed by passing a filename argument.
        '''
        if not prefix:
            prefix = self.dump_prefix
        if self.data is not None:
            self.data.to_csv(prefix+'_genotypes.tsv', sep='\t')
        if self.allele_frequencies is not None:
            self.allele_frequencies.to_csv(prefix+'_allele_frequencies.tsv', sep='\t')

In [3]:
test = VCF('beagle_output/TEST_DATA/TEST.vcf.gz')

In [4]:
test

In [5]:
test.data

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,N00_HITOMEBORE,...,N01_164,N01_165,N01_166,N01_167,N01_168,N01_169,N01_170,N01_171,N01_172,N01_173
0,chr01,1411,.,A,G,.,PASS,.,GT,A,...,A,H,A,A,B,B,H,A,A,A
1,chr01,6918,.,G,T,.,PASS,.,GT,A,...,A,H,A,A,B,B,H,A,A,A
2,chr02,9619,.,T,C,.,PASS,.,GT,A,...,B,A,B,A,B,B,A,A,B,B
3,chr02,36401,.,G,A,.,PASS,.,GT,A,...,B,A,B,A,B,B,A,A,B,B
4,chr03,2137,.,A,G,.,PASS,.,GT,A,...,A,H,B,A,A,A,A,A,A,A
5,chr03,198008,.,G,A,.,PASS,.,GT,A,...,A,H,B,A,H,A,A,A,A,A


In [6]:
test.calculate_allele_frequencies()

,Homozygous A,Homozygous B
0,62,96
1,62,96
2,85,73
3,85,73
4,82,78
5,81,77


In [7]:
test.chisquare()

,Homozygous A,Homozygous B,statistic,pvalue,qvalue,significant
0,62,96,7.316456,0.006833,0.020498,True
1,62,96,7.316456,0.006833,0.020498,True
2,85,73,0.911392,0.339745,0.509618,False
3,85,73,0.911392,0.339745,0.509618,False
4,82,78,0.100000,0.751830,0.751830,False
5,81,77,0.101266,0.750316,0.751830,False


In [8]:
test.dump()